[Torch.Autograd를 사용한 자동 미분](https://tutorials.pytorch.kr/beginner/basics/autogradqs_tutorial.html)

In [2]:
"""
역전파 알고리즘
    매개변수는 주어진 매개변수에 대한 손실 함수의 변화도(gradient)에 따라 조정됨
    변화도 계산을 위한 자동 미분 엔진: torch.autograd
        모든 계산 그래프에 대한 변화도의 자동 계산 지원

예) 입력 x, 매개변수 w와 b, 간단한 단일 계층 신경망
"""

import torch

x = torch.ones(5)
y = torch.zeros(3)
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

## Tensor, Function과 연산그래프

![](https://tutorials.pytorch.kr/_images/comp-graph.png)

w, b는 최적화를 해야 하는 매개변수\
변수들에 대한 손실 함수의 변화도 계산을 위해 `requires_grad` 속성 설정

In [3]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x7fed72612e00>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7fed726114b0>


## 변화도 계산

In [4]:
"""
신경망에서 매개변수의 가중치 최적화를 위해 손실함수의 도함수를 계산
도함수 계산을 위해 loss.backward() 호출한 뒤, w.grad, b.grad에서 값을 가져옴
"""

loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2793, 0.3094, 0.0068],
        [0.2793, 0.3094, 0.0068],
        [0.2793, 0.3094, 0.0068],
        [0.2793, 0.3094, 0.0068],
        [0.2793, 0.3094, 0.0068]])
tensor([0.2793, 0.3094, 0.0068])


## 변화도 추적 멈추기

In [5]:
"""
모델 학습한 뒤 입력 데이터를 단순히 적용하기만 하는 경우의 경우에는 추적 필요 없다
연산 코드를 torch.no_grad() 블록으로 둘러싸서 연산 추적 멈출 수 있다
"""

z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

True
False


In [6]:
# 다른 방법: detach() 메소드 사용
z = torch.matmul(x, w) + b
z_det = z.detach()
print(z_det.requires_grad)

False


## 연산 그래프에 대한 추가 정보

`autograd`는 텐서의 실행된 모든 연산들의 기록을 DAG에 저장\
DAG의 잎은 입력 텐서이고, 뿌리는 결과 텐서\
뿌리에서부터 잎까지 chain rule에 따라 변화도를 자동으로 계산

* 순전파 단계에서 autograd의 작업

1. 요청된 연산 수행하여 결과 텐서 계산
2. DAG에 연산의 변화도 기능을 유지

* 역전파 단계는 DAG의 뿌리에서 `.backward()` 호출시 시작

1. 각 `.grad_fn`으로부터 변화도 계산
2. 각 텐서의 `.grad` 속성에 계산 결과 쌓기
3. 연쇄 법칙 사용해서 모든 잎 텐서들까지 전파

## 선택적으로 읽기: 텐서 변화도와 야코비안 곱

In [12]:
"""
대부분의 경우 스칼라 손실 함수를 이용해서 일부 매개변수와 관련한 변화도 계산

출력 함수가 임의의 텐서인 경우에는?
    PyTorch는 실제 변화도가 아닌 야코비안 곱을 계산
    야코비안 행렬 자체를 계산하는 대신 주어진 입력벡터에 대한 야코비안 곱을 구함
"""

inp = torch.eye(4, 5, requires_grad=True)
out = (inp+1).pow(2).t()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"First call\n{inp.grad}")

# PyTorch가 변화도를 누적시키기 때문에 값이 달라진다
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")

# grad 속성을 0으로 만들어서 변화도 누적 방지
# 실제 학습 과정에서는 옵티마이저가 해당 과정 도와줌
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])
